# PySpark Fifa20
## Practice PySpark SQL w/ FIFA20 datasets
### cr. https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset#players_15.csv

In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.0.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('fifa1').getOrCreate()

In [5]:
# Read CSV file to Dataframe
df = spark.read.csv('players_20.csv',header=True,inferSchema=True)

In [15]:
# Print All columns name and ColumnType
df.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: timestamp (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- nationality: string (nullable = true)
 |-- club: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: integer (nullable = true)
 |-- wage_eur: integer (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- international_reputation: integer (nullable = true)
 |-- weak_foot: integer (nullable = true)
 |-- skill_moves: integer (nullable = true)
 |-- work_rate: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- real_face: string (nullable = true)
 |-- release_clause_eur: integer (nullable = true)
 |-- player_tag

In [36]:
# Liverpool's players overall & potential 
liverpool_players = df.filter(df.club == 'Liverpool').select(['short_name', 'overall', 'potential'])
liverpool_players.show()

+--------------------+-------+---------+
|          short_name|overall|potential|
+--------------------+-------+---------+
|         V. van Dijk|     90|       91|
|            M. Salah|     90|       90|
|             Alisson|     89|       91|
|             S. Mané|     88|       88|
|     Roberto Firmino|     86|       87|
|             Fabinho|     85|       89|
|        A. Robertson|     85|       89|
|        G. Wijnaldum|     84|       84|
| T. Alexander-Arnold|     83|       89|
|        J. Henderson|     83|       83|
|            N. Keïta|     82|       87|
|            J. Matip|     82|       84|
|          X. Shaqiri|     82|       82|
|           J. Milner|     81|       81|
|            J. Gomez|     80|       88|
|A. Oxlade-Chamber...|     80|       83|
|           D. Lovren|     80|       80|
|          A. Lallana|     79|       79|
|            D. Origi|     78|       83|
|            N. Clyne|     78|       78|
+--------------------+-------+---------+
only showing top

In [72]:
# Top 10 fastest pace score for outfield players in Fifa20
outfield_players = df.filter(df.team_position != 'GK')
# drop the 'null' value in the pace column
outfield_players_drop_na = outfield_players.na.drop(subset=['pace'])
outfield_players_drop_na.select(['short_name','pace','player_positions']).orderBy(df['pace'].desc()).show(10)

+--------------+----+----------------+
|    short_name|pace|player_positions|
+--------------+----+----------------+
|  Adama Traoré|  96|         RW, RWB|
|     K. Mbappé|  96|          ST, RW|
|      A. Chalá|  95|          LB, LM|
|      K. Nagai|  95|      ST, LM, LW|
|       L. Sané|  95|              LW|
|Gelson Martins|  95|          RM, RW|
|      D. James|  94|      LM, LW, RM|
|       S. Mané|  94|          LW, LM|
| T. Barkhuizen|  94|          RM, LM|
|       I. Sarr|  94|          RM, LM|
+--------------+----+----------------+
only showing top 10 rows



In [28]:
# Top 10 players who always find the spot to get the score and has a contract less than 2 years
from pyspark.sql.functions import split
split_work_rate = split(df['work_rate'], '/')
wr_df = df.withColumn('attack_work_rate', split_work_rate.getItem(0))
wr_df = wr_df.withColumn('defense_work_rate', split_work_rate.getItem(1))
high_atk_wr_df = wr_df.filter((wr_df['attack_work_rate'] == 'High') & (wr_df['mentality_positioning'] > 80) & ((wr_df['contract_valid_until'] - 2020) < 2))
high_atk_wr_df.select(['short_name','attack_work_rate','mentality_positioning','contract_valid_until']).orderBy(high_atk_wr_df['mentality_positioning'].desc()).show(10)

+--------------+----------------+---------------------+--------------------+
|    short_name|attack_work_rate|mentality_positioning|contract_valid_until|
+--------------+----------------+---------------------+--------------------+
|     S. Agüero|            High|                   93|                2021|
|     E. Cavani|            High|                   93|                2020|
|     T. Müller|            High|                   92|                2021|
|     L. Suárez|            High|                   92|                2021|
|R. Lewandowski|            High|                   91|                2021|
|  M. Mandžukić|            High|                   90|                2021|
|        Falcao|            High|                   90|                2020|
|    G. Higuaín|            High|                   90|                2021|
|   David Silva|            High|                   88|                2020|
| José Callejón|            High|                   88|                2020|